In [1]:
!nvidia-smi

Mon Dec 16 08:12:17 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torchvision.utils import save_image
import csv
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [3]:
class LowDimDataset(Dataset):
    def __init__(self, noisy_dir, gt_dir=None, transform=None):
        self.noisy_images = sorted([os.path.join(noisy_dir, fname) for fname in os.listdir(noisy_dir)])
        self.gt_images = sorted([os.path.join(gt_dir, fname) for fname in os.listdir(gt_dir)]) if gt_dir else None
        self.transform = transform

    def __len__(self):
        return len(self.noisy_images)

    def __getitem__(self, idx):
        noisy_image = Image.open(self.noisy_images[idx]).convert("RGB")
        if self.transform:
            noisy_image = self.transform(noisy_image)
        
        if self.gt_images:
            gt_image = Image.open(self.gt_images[idx]).convert("RGB")
            if self.transform:
                gt_image = self.transform(gt_image)
            return noisy_image, gt_image
        return noisy_image


In [4]:
#Transformations
transform = Compose([  # Ensure all images are of the same size
    ToTensor(),
])
torch.cuda.empty_cache()

In [5]:
class ResidualDenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate=32, num_layers=5):
        super(ResidualDenseBlock, self).__init__()
        self.layers = nn.ModuleList()
        total_channels = in_channels
        for i in range(num_layers):
            out_channels = growth_rate
            layer = nn.Sequential(
                nn.Conv2d(total_channels, out_channels, 3, padding=1),
                nn.LeakyReLU(negative_slope=0.2, inplace=True)
            )
            self.layers.append(layer)
            total_channels += out_channels
        
        # Local Feature Fusion
        self.lff = nn.Conv2d(total_channels, in_channels, 1)
    
    def forward(self, x):
        features = [x]
        for layer in self.layers:
            new_feature = layer(torch.cat(features, dim=1))
            features.append(new_feature)
        return x + self.lff(torch.cat(features, dim=1))

class UpsampleBlock(nn.Module):
    def __init__(self, in_channels, scale_factor=2):
        super(UpsampleBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels * (scale_factor ** 2), 3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(scale_factor)
        self.activation = nn.LeakyReLU(negative_slope=0.2, inplace=True)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.pixel_shuffle(x)
        return self.activation(x)

class RealESRGAN(nn.Module):
    def __init__(self, in_channels=3, base_channels=64, num_blocks=23, upscale_factor=4):
        super(RealESRGAN, self).__init__()
        
        # Initial convolution with noise reduction capability
        self.initial_noise_reduction = nn.Sequential(
            nn.Conv2d(in_channels, base_channels//2, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Conv2d(base_channels//2, base_channels, kernel_size=3, padding=1),
            nn.LeakyReLU(negative_slope=0.2)
        )
        
        # Initial feature extraction
        self.first_conv = nn.Conv2d(base_channels, base_channels, 3, padding=1)
        
        # Residual blocks
        self.residual_blocks = nn.ModuleList([
            ResidualDenseBlock(base_channels) for _ in range(num_blocks)
        ])
        
        # Skip connection
        self.skip_conv = nn.Conv2d(base_channels, base_channels, 3, padding=1)
        
        # Multiple upsampling stages for 4x scaling
        self.upsample = nn.Sequential(
            UpsampleBlock(base_channels, scale_factor=2),
            UpsampleBlock(base_channels, scale_factor=2)
        )
        
        # Final refinement conv
        self.refinement = nn.Sequential(
            nn.Conv2d(base_channels, base_channels//2, 3, padding=1),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Conv2d(base_channels//2, in_channels, 3, padding=1)
        )
    
    def forward(self, x):
        # Initial noise reduction
        x = self.initial_noise_reduction(x)
        
        # Initial feature extraction
        features = self.first_conv(x)
        skip = features
        
        # Residual blocks
        for block in self.residual_blocks:
            features = block(features)
        
        # Skip connection
        features += self.skip_conv(skip)
        
        # Upsampling
        features = self.upsample(features)
        
        # Final refinement
        return self.refinement(features)


In [6]:
def train_model(model, train_loader, val_loader, criterion, optimizer, 
                num_epochs=10, device=None, patience=5, 
                models_dir='/kaggle/working/models'):
    
    device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, 
                                   patience=2, verbose=True)
    

    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    epochs_no_improve = 0
    
    
    os.makedirs(models_dir, exist_ok=True)
    
    # Training loop
    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        for noisy, gt in progress_bar:
            # Move data to device
            noisy = noisy.to(device)
            gt = gt.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(noisy)
            
            # Compute loss
            loss = criterion(outputs, gt)
            
            # Backward pass
            loss.backward()
            
            # Optimize
            optimizer.step()
            
            # Update progress bar and track loss
            train_loss += loss.item()
            progress_bar.set_postfix({'Loss': loss.item()})
        
        # Average training loss for epoch
        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for noisy, gt in tqdm(val_loader, desc='Validation'):
                noisy = noisy.to(device)
                gt = gt.to(device)
                
                outputs = model(noisy)
                loss = criterion(outputs, gt)
                
                val_loss += loss.item()
        
        # Average validation loss
        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        # Learning rate scheduling
        scheduler.step(avg_val_loss)
        
        # Print epoch summary
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        
        # Model checkpointing
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            
            # Save best model
            best_model_path = os.path.join(models_dir, 'i_model.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_losses': train_losses,
                'val_losses': val_losses,
                'best_val_loss': best_val_loss
            }, best_model_path)
            print(f'Saved new best model with val loss: {best_val_loss:.4f}')
        else:
            epochs_no_improve += 1
        
        # Early stopping
        if epochs_no_improve >= patience:
            print(f'Early stopping triggered after {epoch+1} epochs')
            break
        
        # Visualize training progress
        plt.figure(figsize=(10, 5))
        plt.plot(train_losses, label='Training Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(models_dir, 'loss_plot.png'))
        plt.close()
    
    # Save final model
    final_model_path = os.path.join(models_dir, 'final_model.pth')
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_losses': train_losses,
        'val_losses': val_losses
    }, final_model_path)
    
    return model, train_losses, val_losses

In [7]:
#Dataset dir
train_train_dir = "/kaggle/input/enhance-the-dark-world/archive/train/train"
train_gt_dir = "/kaggle/input/enhance-the-dark-world/archive/train/gt"
val_val_dir = "/kaggle/input/enhance-the-dark-world/archive/val/val"
val_gt_dir = "/kaggle/input/enhance-the-dark-world/archive/val/gt"
test_dir = "/kaggle/input/enhance-the-dark-world/archive/test"

In [8]:
train_dataset = LowDimDataset(train_train_dir, train_gt_dir, transform=transform)
val_dataset = LowDimDataset(val_val_dir, val_gt_dir, transform=transform)
test_dataset = LowDimDataset(test_dir, transform=transform)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [10]:
model = RealESRGAN(in_channels=3, base_channels=64, num_blocks=23, upscale_factor=4)
# loss function
criterion = nn.MSELoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [11]:
# Train the model
trained_model, train_losses, val_losses = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=15,
    patience=5
    )

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Validation: 100%|██████████| 267/267 [00:28<00:00,  9.42it/s]


Epoch 1/15:
Train Loss: 0.0009, Val Loss: 0.0002
Saved new best model with val loss: 0.0002


Validation: 100%|██████████| 267/267 [00:25<00:00, 10.61it/s]


Epoch 2/15:
Train Loss: 0.0004, Val Loss: 0.0002
Saved new best model with val loss: 0.0002


Validation: 100%|██████████| 267/267 [00:25<00:00, 10.35it/s]


...

Epoch 13/15:
Train Loss: 0.0003, Val Loss: 0.0002


Validation: 100%|██████████| 267/267 [00:24<00:00, 11.04it/s]


Epoch 14/15:
Train Loss: 0.0003, Val Loss: 0.0002
Saved new best model with val loss: 0.0002


Validation: 100%|██████████| 267/267 [00:24<00:00, 10.89it/s]


Epoch 15/15:
Train Loss: 0.0003, Val Loss: 0.0002


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
output_dir = "/kaggle/working/test_gt"
os.makedirs(output_dir, exist_ok=True)

model.eval()
with torch.no_grad():
    for idx, noisy in enumerate(test_loader):
        noisy = noisy.to(device)
        outputs = model(noisy)
        image_name = f"gt_{idx+1:05d}.png"  # Image name
        save_image(outputs, os.path.join(output_dir, image_name))
        #print(f"Saved image: {image_name}")  # Print image name
print('All Images saved in .png format')

All Images saved in .png format


In [14]:
def images_to_csv(folder_path, output_csv):
    data_rows = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L')
            #print(image.size)
            image_array = np.array(image).flatten()[::8]
            #print(image_array.shape)
            # Replace 'test_' with 'gt_' in the ID
            image_id = filename.split('.')[0]
            data_rows.append([image_id, *image_array])
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')

folder_path = '/kaggle/working/test_gt'
output_csv = 'submission.csv'
images_to_csv(folder_path, output_csv)


Successfully saved to submission.csv


In [15]:
sub = pd.read_csv("/kaggle/working/submission.csv")
sub.head()

,ID,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_81910,pixel_81911,pixel_81912,pixel_81913,pixel_81914,pixel_81915,pixel_81916,pixel_81917,pixel_81918,pixel_81919
0,gt_00001,35,57,59,48,52,56,48,45,47,...,28,28,28,28,27,26,28,28,28,27
1,gt_00002,26,38,45,50,50,44,38,35,40,...,33,33,36,37,36,34,33,34,36,36
2,gt_00003,25,38,38,36,38,36,34,37,42,...,33,35,38,38,36,36,39,39,39,36
3,gt_00004,31,58,62,61,63,68,78,99,98,...,42,42,42,41,41,43,43,42,43,43
4,gt_00005,20,25,27,29,26,25,24,25,24,...,51,48,47,47,50,50,51,52,51,49
